In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
URL = "https://www.amazon.com/s?k=iPhone+12+Pro+Max&ref=nb_sb_noss_2"
HEADERS = {
        'Accept-Language': 'en-US, en;q=0.5',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'
    }

# HTTP Request
webpage = requests.get(URL, headers=HEADERS)
# Connection 
print(requests.get(URL, headers=HEADERS))

<Response [200]>


In [3]:
# Function to extract product information
def extract_product_info(link):
    HEADERS = {
        'Accept-Language': 'en-US, en;q=0.5',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'
    }
    # HTTP Request
    webpage = requests.get(link, headers=HEADERS)
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    # Extracting required information
    title = soup.find("span", attrs={"id": 'productTitle'}).text.strip()
    rating = soup.find("span", attrs={"class": 'a-icon-alt'}).text
    div_element = soup.find("div", attrs={"id": "imgTagWrapperId"})
    img_tag = div_element.find("img")
    img_src = img_tag.get("src")
    return {'Title': title, 'Rating': rating, 'Image URL': img_src}

# Function to extract product links from a search page
def extract_product_links(URL):
    HEADERS = {'Accept-Language': 'en-US, en;q=0.5',
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    # Fetching links
    links = soup.find_all("a", class_="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal")
    product_links = ["https://amazon.com" + link.get('href') for link in links]
    return product_links

# Function to get total number of pages
def get_total_pages(URL):
    HEADERS = {'Accept-Language': 'en-US, en;q=0.5',
               'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36 Edg/121.0.0.0'}
    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)
    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")
    # Finding pagination element
    pagination = soup.find('div', {'class': 's-pagination-container'})
    if pagination:
        # Extracting last page number
        last_page_element = pagination.find('span', {'class': 's-pagination-item s-pagination-disabled'})
        if last_page_element:
            last_page_number = int(last_page_element.text)
            return last_page_number
        else:
            return 1  # If only one page
    else:
        return 1  # If pagination is not found, assume only one page

In [4]:
# Main function to perform crawling
def main():
    base_url = "https://www.amazon.com/s?k=iPhone+12+Pro+Max&ref=nb_sb_noss_2"
    total_pages = get_total_pages(base_url)
    all_products = []
    
    for page_num in range(1, total_pages + 1):
        page_url = f"{base_url}&page={page_num}"
        product_links = extract_product_links(page_url)
        for link in product_links:
            product_info = extract_product_info(link)
            all_products.append(product_info)
    
    # Converting to DataFrame
    df = pd.DataFrame(all_products)
    print(df)
    # Store in a CSV file
    df.to_csv('crawling_amazon_data.csv', index=False)

In [5]:
if __name__ == "__main__":
    main()

                                                 Title              Rating  \
0    JETech Case for iPhone 12 Pro Max 6.7-Inch, No...  4.6 out of 5 stars   
1    OTTERBOX COMMUTER SERIES Case for iPhone 12 Pr...  4.6 out of 5 stars   
2    OtterBox Defender Series Screenless Edition Ca...  4.7 out of 5 stars   
3    OtterBox iPhone 13 Pro Max & iPhone 12 Pro Max...  4.5 out of 5 stars   
4    OtterBox iPhone 13 Pro Max and iPhone 12 Pro M...  4.5 out of 5 stars   
..                                                 ...                 ...   
339  iPhone 12 Pro Max Lepricorn Funny Leprechaun R...  5.0 out of 5 stars   
340  Motorola razr | 2023 | Unlocked | Made for US ...  4.3 out of 5 stars   
341  Motorola razr+ | 2023 | Unlocked | Made for US...  4.1 out of 5 stars   
342  iPhone 12 Pro Max Emerald Green Shamrock Irish...  5.0 out of 5 stars   
343  iPhone 12 Pro Max Emerald Green Shamrock Irish...  5.0 out of 5 stars   

                                             Image URL  
0    h

In [6]:
import requests
from bs4 import BeautifulSoup


# Test the function
URL = "https://www.amazon.com/s?k=iPhone+12+Pro+Max&ref=nb_sb_noss_2"
total_pages = get_total_pages(URL)
print("Total pages:", total_pages)


Total pages: 20
